## Experiment 1 
- Seeing which algorithm converges faster given a particular unitary size and given resolution 
- Resolution ranges from [10,50] with a changed of 5 in between 
- Max iterations is 10 and error threshold is $10^{-3}$

In [1]:
resolutions = [i for i in range(10, 55, 5)]
resolutions

[10, 15, 20, 25, 30, 35, 40, 45, 50]

In [2]:
from qiskit import IBMQ
from qiskit import QuantumCircuit, execute, transpile, Aer
from qiskit.extensions import UnitaryGate, Initialize
from qiskit.quantum_info import Statevector
from qiskit.tools.visualization import plot_bloch_vector
from qiskit.tools.visualization import plot_histogram, plot_bloch_multivector
import numpy as np
from time import sleep
import sys
sys.path.append("../..")
import os
from scipy.stats import unitary_group
import matplotlib.pyplot as plt
%matplotlib inline

# IBMQ.load_account()
# provider = IBMQ.get_provider(hub='ibm-q-education')
# santiago = provider.get_backend('ibmq_santiago')
# casablanca = provider.get_backend('ibmq_casablanca')
# bogota = provider.get_backend('ibmq_bogota')
sim = Aer.get_backend('qasm_simulator')
# athens = provider.get_backend('ibmq_athens')

In [3]:
from Modules.normal_SPEA import SPEA
from Modules.changed_SPEA import global_max_SPEA

### Utils

In [4]:
def generate_plots(unitary_size, costs, errors, overlaps, algorithm):
    import random
    colors = ['red', 'brown', 'cyan', 'green',
              'grey', 'blue', 'purple', 'black', 'orange']
    c1, c2, c3 = random.sample(colors, 3)

    # plot
    os.makedirs("Experiment_1/"+str(unitary_size) +
                "_qubit(random)/", exist_ok=True)
    # plot 1
    fig = plt.figure(figsize=(13, 6))
    ax1 = fig.add_subplot(1, 2, 1)
    ax1.set_title(str(unitary_size)+" qubit "+algorithm +
                  " Cost v/s Max iters", fontsize=16)
    ax1.set_xlabel("Number of Resolutions ", fontsize=15)
    ax1.set_ylabel("Metrics Returned for unitary ", fontsize=15)
    ax1.plot(resolutions, costs, label='Costs of Unitary',
             marker='o', color=c1, alpha=0.7)
    ax1.plot(resolutions, overlaps, label='Average overlap from nearest eigenvector',
             marker='s', color=c2, alpha=0.6)
    ax1.legend(loc='best')
    ax1.grid()
    # plot 2
    ax2 = fig.add_subplot(1, 2, 2)
    ax2.set_title(str(unitary_size)+" qubit "+algorithm +
                  " % error v/s Max iters", fontsize=16)
    ax2.set_xlabel("Number of resolutions ", fontsize=15)
    ax2.set_ylabel("% error for nearest eigenvalue", fontsize=15)
    ax2.plot(resolutions, errors, label='Average error from nearest eigenvalue',
             marker='o', color=c3, alpha=0.6)
    ax2.legend(loc='best')
    ax2.grid()
    # save axure
    fig.savefig("Experiment_1/"+str(unitary_size)+"_qubit(random)/" +
                algorithm+" Algorithm (alternate).JPG", dpi=200)

In [5]:
def get_results(eig_vals, eig_vect, bases, basis_indices, unitary, algorithm, experiments):
    '''Return the results of running the algorithm for this particular unitary matrix'''
    costs_g = []
    errors_g = []
    max_overlaps_g = []
    # find how the cost converges with increasing iterations
    for reso in resolutions:
        costs = []
        errors = []
        overlaps = []
        i = 0
        # run the experiments ...
        while len(costs) < experiments:
            if algorithm == 'original':
                spea = SPEA(unitary, resolution=reso, error=3, max_iters=10)
            else:
                spea = global_max_SPEA(
                    unitary, resolution=reso, error=3, max_iters=10)

            result = spea.get_eigen_pair(
                progress=False, backend=sim, algo='alternate', basis=bases[i], basis_ind=basis_indices[i],
                randomize=False,shots = 2**12)

#             if result['cost'] < 0.65:
#                 continue
                
            # increment the basis index 
            i+=1  # in exp 1 -> basis[0], in exp 2 -> basis[1] and so on....
            
            # find the costs
            costs.append(result['cost'])
            theta = result['theta']
            res_state = result['state']

            # find the abs difference in this theta with the closest eigenvalue
            # and append that to the errors ...
            min_error = 1e5
            for e in eig_vals:
                error = abs(e - theta)
                if error < min_error:
                    min_error = error
                    perc_error = ((error)/e)*100
            errors.append(perc_error)

            # find overlaps
            max_overlap = -1
            for k in eig_vect:
                dot = np.linalg.norm(np.dot(k, res_state.conjugate().T))**2
                max_overlap = max(max_overlap, dot)
            overlaps.append(max_overlap)
        
        print("Result with", reso, " resolutions :")
        print("AVG. COST :", np.average(costs),
              "AVG. ERROR :", np.average(errors))
        # append the average result of your algorithm ...
        costs_g.append(np.average(costs))
        errors_g.append(np.average(errors))
        max_overlaps_g.append(np.average(overlaps))

    return costs_g, errors_g, max_overlaps_g

## 2 - qubit unitary


In [10]:
unit_2 = unitary_group.rvs(4)
unit_2

array([[ 0.42301886+1.21660658e-01j, -0.65963338+3.73554408e-01j,
        -0.18525662-3.36202564e-04j, -0.42228448-1.37660205e-01j],
       [-0.64298263-7.13600650e-05j,  0.00808872+3.00568579e-01j,
         0.26424576-4.30078301e-01j, -0.42757007-2.41985764e-01j],
       [ 0.09090023-4.01928229e-01j, -0.19042309-5.09608118e-01j,
         0.17662822-2.54866545e-01j, -0.42167745+5.10159877e-01j],
       [ 0.0320143 +4.71161329e-01j, -0.15710225+1.19547105e-01j,
         0.7512439 +2.24421083e-01j,  0.07758884+3.42428398e-01j]])

In [11]:
eig_vals2, eig_vect2 = np.linalg.eig(unit_2)
eig_vals2 = np.angle(eig_vals2)
e = []
for k in eig_vals2:
    if k < 0:
        v = (k + 2*np.pi)/(2*np.pi)
    else:
        v = (k)/(2*np.pi)
    e.append(v)
eig_vals2 = np.array(e)
print("Eigenstates :", eig_vect2)
print("Eigenvalues :", eig_vals2)

Eigenstates : [[ 0.52650795-0.06310447j  0.21056612-0.14875779j  0.79640627+0.j
  -0.0981045 +0.09193908j]
 [ 0.66274237+0.j          0.28132978+0.19922114j -0.47395219-0.22650573j
  -0.27267107-0.30274082j]
 [-0.09738967+0.15331384j  0.76540519+0.j         -0.1221906 -0.13716539j
   0.41854012+0.41502325j]
 [ 0.31454626-0.3842532j  -0.21768008+0.42599039j -0.01695622+0.23615488j
   0.68447138+0.j        ]]
Eigenvalues : [0.39374559 0.75352756 0.99946376 0.17598498]


### Generate Basis set

In [12]:
bases2 , basis_indices2 = [], []
for _ in range(4):
    sample = unitary_group.rvs(4)
    basis = []
    for k in sample:
        basis.append(np.array(k, dtype=complex))
    ind = np.random.choice(range(4))
    bases2.append(basis)
    basis_indices2.append(ind)
print("Basis set :",bases2)
print("Basis indices :",basis_indices2)

Basis set : [[array([ 0.48981882+0.15298201j, -0.26666321+0.51410535j,
        0.11562013+0.19072544j, -0.4984273 -0.32107082j]), array([ 0.07760449-0.07568072j, -0.29461687-0.28498114j,
        0.60698511-0.6521419j , -0.12415838-0.10536281j]), array([-0.17083986+0.78881862j,  0.32737969-0.07448516j,
        0.0180167 -0.17376859j, -0.38937447+0.23177755j]), array([ 0.27086009+0.01173636j,  0.61932291+0.01693214j,
       -0.14139571-0.32561163j,  0.19489482-0.61534529j])], [array([-0.6065911 +0.49551241j, -0.12611167+0.29228041j,
        0.33866586+0.09231937j,  0.3853081 +0.11620213j]), array([ 0.43630048+0.2064912j ,  0.09552736+0.05507663j,
        0.35342332+0.6175565j , -0.18716143+0.46209403j]), array([-0.32474628-0.03985988j,  0.68916108+0.13652098j,
        0.16158226-0.29181325j, -0.48157728+0.23704343j]), array([ 0.16517718+0.13851172j, -0.54007379+0.31819559j,
       -0.06845901-0.50452757j, -0.29482859+0.46308128j])], [array([-0.16637693+0.07437035j, -0.27177586+0.28326414

### Algorithm 1

In [ ]:
costs_2qubit_b, errors_eig_2qubit_b, max_overlaps_2qubit_b = get_results(
    eig_vals2, eig_vect2, bases2, basis_indices2, unit_2, 'original', 4)
generate_plots(2, costs_2qubit_b, errors_eig_2qubit_b,
               max_overlaps_2qubit_b, "Original")

### Algorithm 2

In [ ]:
costs_2qubit_c, errors_eig_2qubit_c, max_overlaps_2qubit_c = get_results(
    eig_vals2, eig_vect2, bases2, basis_indices2, unit_2, 'modified', 4)
generate_plots(2, costs_2qubit_c, errors_eig_2qubit_c,
               max_overlaps_2qubit_c, "Modified")

## 3 - qubit unitary

In [ ]:
unit_3 = unitary_group.rvs(8)
unit_3

In [ ]:
eig_vals3, eig_vect3 = np.linalg.eig(unit_3)
eig_vals3 = np.angle(eig_vals3)
e = []
for k in eig_vals3:
    if k < 0:
        v = (k + 2*np.pi)/(2*np.pi)
    else:
        v = (k)/(2*np.pi)
    e.append(v)
eig_vals3 = np.array(e)
print("Eigenstates :", eig_vect3)
print("Eigenvalues :", eig_vals3)

### Generate Basis

In [ ]:
bases3 , basis_indices3 = [], []
for _ in range(4):
    sample = unitary_group.rvs(8)
    basis = []
    for k in sample:
        basis.append(np.array(k, dtype=complex))
    ind = np.random.choice(range(8))
    bases3.append(basis)
    basis_indices3.append(ind)
print("Basis indices :",basis_indices3)

- Algorithm 1

In [ ]:
costs_3qubit_b, errors_eig_3qubit_b, max_overlaps_3qubit_b = get_results(
    eig_vals3, eig_vect3, bases3, basis_indices3, unit_3, 'original', 4)

In [ ]:
generate_plots(3, costs_3qubit_b, errors_eig_3qubit_b,
               max_overlaps_3qubit_b, "Original")

- Algorithm 2

In [ ]:
costs_3qubit_c, errors_eig_3qubit_c, max_overlaps_3qubit_c = get_results(
    eig_vals3, eig_vect3, bases3, basis_indices3, unit_3, 'modified', 4)
generate_plots(3, costs_3qubit_c, errors_eig_3qubit_c,
               max_overlaps_3qubit_c, "Modified")

## 4 - qubit unitary

In [ ]:
unit_4 = unitary_group.rvs(16)
# unit_4

In [ ]:
eig_vals4, eig_vect4 = np.linalg.eig(unit_4)
eig_vals4 = np.angle(eig_vals4)
e = []
for k in eig_vals4:
    if k < 0:
        v = (k + 2*np.pi)/(2*np.pi)
    else:
        v = (k)/(2*np.pi)
    e.append(v)
eig_vals4 = np.array(e)
print("Eigenstates :", eig_vect4)
print("Eigenvalues :", eig_vals4)

### Generate basis set

In [ ]:
bases4 , basis_indices4 = [], []
for _ in range(4):
    sample = unitary_group.rvs(16)
    basis = []
    for k in sample:
        basis.append(np.array(k, dtype=complex))
    ind = np.random.choice(range(16))
    bases4.append(basis)
    basis_indices4.append(ind)
print("Basis indices :",basis_indices4)

- Algorithm 1

In [ ]:
costs_4qubit_b, errors_eig_4qubit_b,  max_overlaps_4qubit_b = get_results(
    eig_vals4, eig_vect4, bases4, basis_indices4, unit_4, 'original', 4)
generate_plots(4, costs_4qubit_b, 
               errors_eig_4qubit_b, max_overlaps_4qubit_b,  "Original")

- Algorithm 2

In [ ]:
costs_4qubit_c, errors_eig_4qubit_c, max_overlaps_4qubit_c = get_results(
    eig_vals4, eig_vect4, bases4, basis_indices4, unit_4, 'modified', 4)
generate_plots(4, costs_4qubit_c, 
               errors_eig_4qubit_c,max_overlaps_4qubit_c, "Modified")